In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("../data/data_train.csv", index_col=0)
data_company_response_binary = data[~data["Company response"].isin(["In progress", "Untimely response"])]
estado_respuesta = {'Closed with monetary relief': 0, 
       'Closed with non-monetary relief':0, 'Closed with explanation': 1, 'Closed': 1}

data_company_response_binary["Company response"] = data_company_response_binary["Company response"].map(estado_respuesta)


X = data_company_response_binary.drop(columns=["Company response"])
y = data_company_response_binary["Company response"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

C:\Users\Fernando\AppData\Local\Temp\ipykernel_5604\2107138701.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_company_response_binary["Company response"] = data_company_response_binary["Company response"].map(estado_respuesta)


# PIPELINE

In [3]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from gensim.models import Word2Vec
import sys
sys.path.append("C:/Users/Fernando/Desktop/Proyecto_Final_ML/app")
from utils import *
import numpy as np
from sklearn.linear_model import LogisticRegression
import joblib

pipeline = Pipeline([
    ("fill_na", filling_na),
    ("encoder", FillEncodeColumn(column="Sub-product")),
    ("date_transformation_received", date_converter_received),
    ("date_transformation_sent", date_converter_sent),
    ("day_week_received", day_of_week_month_received),
    ("day_week_sent", day_of_week_month_sent),
    ("product_encoder", ProductEncoder(column="Product")),
    ("reg_and_div", regions_and_divisions),
    ("red_enc", RegionDivisionEncoder(column="regions")),
    ("div_enc", RegionDivisionEncoder(column="divisions")),
    ("cleaner text", text_cleaner),
    ("embedding creation", Word2VecVectorizer(column="Issue_tokens")),
    ("company type", company_type_converter_pipe),
    ("company from porduct", Company_type_from_product(column="Company_type")),
    ("Drop columns", ColumnDropper(columns_to_drop=["Complaint ID","Sub-issue", "ZIP code", "Company", 
             "Timely response?", "Consumer disputed?"])),
    ("model", LogisticRegression(C= 10, penalty='l2', solver= 'liblinear', class_weight="balanced", random_state=42))

])

pipeline.fit(X_train, y_train)

joblib.dump(pipeline, "C:/Users/Fernando/Desktop/Proyecto_Final_ML/app/pipeline_produccion.pkl" )

['C:/Users/Fernando/Desktop/Proyecto_Final_ML/app/pipeline_produccion.pkl']